This file is used to build all of the json data that gets loaded by `train.ipynb`

In [ ]:
# Create palette file from palette.png
from util.convert_image_to_palette import convert 

convert("data/palette.png", "data/palette.json")

In [ ]:
# Convert all files in the training .vox folder to json files
import os
from util.convert_vox_to_json import convert_vox_to_json

filenames = os.listdir('training/vox')

for filename in filenames:
    if "vox" in filename:
        new_filename = filename.replace("vox", "json")
        convert_vox_to_json("training/vox/" + filename, "training/json/" + new_filename)

In [2]:
# Convert training json files to vox files for viewing
import os
from util.convert_vox_to_json import convert_json_to_vox

filenames = os.listdir('training/json')

for filename in filenames:
    if "json" in filename:
        new_filename = filename.replace("json", "vox")
        convert_json_to_vox("training/json/" + filename, "training/view_vox/" + new_filename)

In [14]:
# Convert json files in examples to vox files for viewing
import os
from util.convert_vox_to_json import convert_json_to_vox

filenames = os.listdir('examples/json')

for filename in filenames:
    if "json" in filename:
        new_filename = filename.replace("json", "vox")
        convert_json_to_vox("examples/json/" + filename, "examples/view_vox/" + new_filename)

In [ ]:
# Delete all example files
import os

json_dir = "examples/json/"
json_files = os.listdir(json_dir)

vox_dir = "examples/view_vox/"
vox_files = os.listdir(vox_dir)

for item in json_files:
    if item.endswith(".json"):
        os.remove(os.path.join(json_dir, item))

for item in vox_files:
    if item.endswith(".vox"):
        os.remove(os.path.join(vox_dir, item))

In [1]:
import training
import random
import json
import os

# Delete existing sorpok files
sorpok_dir = "training/json/"
sorpok_files = os.listdir(sorpok_dir)

for item in sorpok_files:
    if "sorpok" in item:
        os.remove(os.path.join(sorpok_dir, item))

# Generate sorpok files
sorpok_count = 50

# Light Blue: 246
# Blue: 231
# Yellow: 252
# Green: 47
# White: 15
# Red: 192

# Set up parameters
for i in range(sorpok_count):
    size = 4
    r = random.random()
    if r < 0.5:
        color1 = 246
        color2 = 231
    else:
        color1 = 15
        color2 = 192
    offset = (int(random.random() * size), int(random.random() * size), int(random.random() * size))
    data = {
        "size": {
            "x": training.SIZE[0],
            "y": training.SIZE[1],
            "z": training.SIZE[2],
        },
        "voxels": {},
    }

    for x in range(training.SIZE[0]):
        for y in range(training.SIZE[1]):
            for z in range(training.SIZE[2]):
                # Apply offset
                cur_pos = (x, y, z)
                offset_pos = training.add(cur_pos, offset)
                offset_pos = (offset_pos[0] % size, offset_pos[1] % size, offset_pos[2] % size)

                # Build lattice structure
                if offset_pos[0] == 0 and offset_pos[1] == 0 and offset_pos[2] == 1:
                    data["voxels"][training.ttos(cur_pos)] = color1
                else:
                    c = 0
                    for pos in offset_pos:
                        if pos == 0:
                            c += 1
                    if c >= 2:
                        data["voxels"][training.ttos(cur_pos)] = color2

    # Save JSON file
    output_filename = f'training/json/sorpok{i}.json'
    with open(output_filename, 'w') as output_file:
        json.dump(data, output_file, indent=2)
